In [ ]:
import os
import sys
from collections import defaultdict
from datetime import datetime
from pathlib import Path

import openpyxl
from openpyxl.utils import get_column_letter

In [ ]:
# Get First Xlsx in current dir
input_filename = list(
    filter(None, [s if s.is_file() else None for s in Path(".").glob("*.xlsx")])
)[0]
input_filename

In [ ]:
def RemoveBorder(sheet, cell):
    sheet[cell].border = openpyxl.styles.borders.Border()


def AddBorder(sheet, cell):
    side = openpyxl.styles.borders.Side(style="thin")
    sheet[cell].border = openpyxl.styles.borders.Border(
        top=side, bottom=side, left=side, right=side
    )


def RemoveFillColor(sheet, cell):
    sheet[cell].fill = openpyxl.styles.fills.PatternFill()


def AddFillColor(sheet, cell, color):
    sheet[cell].fill = openpyxl.styles.fills.PatternFill(
        start_color=color, end_color=color, fill_type="solid"
    )


def set_number_format(sheet, col, row):
    _cell = sheet.cell(column=col, row=row)
    _cell.number_format = "0"


def colorClearRange(sheet, start_row, end_row, start_col, end_col):
    for r in range(start_row, end_row):
        for c in range(start_col, end_col):
            RemoveFillColor(sheet, "{}{}".format(get_column_letter(c), r))


def colorFillRange(sheet, start_row, end_row, start_col, end_col, color):
    for r in range(start_row, end_row):
        for c in range(start_col, end_col):
            AddFillColor(sheet, "{}{}".format(get_column_letter(c), r), color)

In [ ]:
replacements = {
    "General": {
        "": "Unknown",
        "U": "Unknown",
        "Uk": "Unknown",
        "Unk": "Unknown",
        "Unkn": "Unknown",
        "Unkno": "Unknown",
        "Unknow": "Unknown",
        "Na": "Unknown",
    },
    "Gender": {"Male": "M", "Female": "F"},
    "Ethnicity": {
        "Nh": "Non-Hispanic",
    },
    "Race": {"W": "White", "B": "Black"},
}


def guessBetterName(category, item):
    category = category.capitalize()
    if item in replacements["General"]:
        return replacements["General"][item]
    if category in replacements:
        if item in replacements[category]:
            return replacements[category][item]
    return item


def tally(rows, key):
    tally = defaultdict(int)
    for row in rows:
        if key not in row:
            addTo = "Unknown"
        else:
            addTo = row[key]
            if type(addTo) == str:
                addTo = addTo.strip()
                addTo = addTo.capitalize()
                addTo = guessBetterName(key, addTo)
        tally[addTo] += 1
    return dict(tally)

In [ ]:
class baseProcessor:
    max_column = 0
    rows = []
    log_blanks = False

    def __init__(self, sheet):
        self.sheet = sheet

    def getRows(self):
        return self.rows

    def getExtraSummary(self, rows):
        return {}

    def printExtras(self, summary):
        pass

    def getSummary(self):
        return {}

    def updateSummary(self, new_summary):
        return False

    def generateSummary(rows):
        return {}

In [ ]:
def get_sheet_by_name(xfile, title, create=False):
    for e in xfile.worksheets:
        if e.title == title:
            return e
    if create:
        return xfile.create_sheet(title, 99999)

In [ ]:
class HeaderedFileProcessor(baseProcessor):
    def getHeaders(self):
        out = {}
        for col in range(1, self.max_column + 1):
            letter = get_column_letter(col)

            celldata = self.sheet["{}{}".format(letter, 1)].value
            if celldata is not None:
                out[letter] = celldata.lower()

        return out

    def getDataRowCount(self):
        emptyRowsInRow = 0
        for r in range(1, self.sheet.max_row):
            rowEmpty = True
            for c in range(1, self.max_column + 1):
                letter = get_column_letter(c)
                celldata = self.sheet["{}{}".format(letter, r)].value
                if celldata is not None:
                    rowEmpty = False
            if rowEmpty:
                emptyRowsInRow += 1
            else:
                emptyRowsInRow = 0
            if emptyRowsInRow == 2:
                return r - 1
        return self.sheet.max_row + 1

    def _retitleRowDict(self, headers, row):
        out = {}
        for item in headers:
            if item in row:
                out[headers[item]] = row[item]
            else:
                out[headers[item]] = "Unknown"
        return out

    def getRow(self, r):
        row = {}
        for c in range(1, self.max_column + 1):
            letter = get_column_letter(c)
            celldata = self.sheet["{}{}".format(letter, r)].value
            if celldata is not None:
                row[letter] = celldata
            elif self.log_blanks:
                print("\t", "Blank Cell Located")
                print("\t", self.sheet["{}{}".format(letter, r)])
        return row

    def ConvertRowsToList(self):
        out = []
        headers = self.getHeaders()
        for r in range(2, self.getDataRowCount()):
            row = self.getRow(r)
            out.append(self._retitleRowDict(headers, row))

        return out

    def getMaxColumn(self):
        for col in range(1, 30):
            celldata = self.sheet["{}{}".format(get_column_letter(col), 1)].value
            if celldata is None:
                return col - 1

    def __init__(self, sheet):
        self.sheet = sheet
        self.max_column = self.getMaxColumn()

In [ ]:
class basicProcessor(HeaderedFileProcessor):
    containsExistingSummary = False
    summary_cols = [
        "Genders",
        "Age Ranges",
        "Ethnicity",
        "Race",
        "Zip Codes",
    ]

    @staticmethod
    def CanProcess(sheet):
        emptyRowsInRow = 0
        for r in range(1, sheet.max_row):
            celldata = sheet["{}{}".format("A", r)].value
            if celldata == None:
                emptyRowsInRow += 1
            if emptyRowsInRow == 2:
                containsExistingSummary = True
        return True

    def getSummaryRow(self):
        emptyRowsInRow = 0
        for r in range(1, self.sheet.max_row):
            celldata = self.sheet["{}{}".format("A", r)].value
            if celldata == None:
                emptyRowsInRow += 1

            if emptyRowsInRow == 1:
                return r
        return self.sheet.max_row + 2

    def getSummary(self):
        summary = {}
        rows = self.ConvertRowsToList()
        ageTally = defaultdict(int)
        for row in rows:
            if type(row["age"]) == int:
                if row["age"] >= 0 and row["age"] <= 11:
                    ageTally["0-11"] += 1

                if row["age"] >= 12 and row["age"] <= 19:
                    ageTally["12-19"] += 1

                if row["age"] >= 20 and row["age"] <= 29:
                    ageTally["20-29"] += 1

                if row["age"] >= 30 and row["age"] <= 39:
                    ageTally["30-39"] += 1

                if row["age"] >= 40 and row["age"] <= 49:
                    ageTally["40-49"] += 1

                if row["age"] >= 50 and row["age"] <= 59:
                    ageTally["50-59"] += 1

                if row["age"] >= 60 and row["age"] <= 69:
                    ageTally["60-69"] += 1

                if row["age"] >= 70 and row["age"] <= 79:
                    ageTally["70-79"] += 1
                if row["age"] >= 80:
                    ageTally["80+"] += 1

        summary["Age Ranges"] = dict(ageTally)
        headers = list(self.getHeaders().values())
        if "zip code" in headers:
            summary["Zip Codes"] = tally(rows, "zip code")
        if "ethnicity" in headers:
            summary["Ethnicity"] = tally(rows, "ethnicity")
        if "gender" in headers:
            summary["Genders"] = tally(rows, "gender")
        if "race" in headers:
            summary["Race"] = tally(rows, "race")
        summary.update(self.getExtraSummary(rows))
        return summary

    def clearSummary(self):
        for r in range(self.summaryRow, self.sheet.max_row + 1):
            for c in range(1, self.sheet.max_column + 1):
                letter = get_column_letter(c)
                pos = "{}{}".format(letter, r)
                self.sheet[pos].value = ""
                RemoveFillColor(self.sheet, pos)
                RemoveBorder(self.sheet, pos)

    def updateSummary(self, summary):
        c = 1
        for category in self.summary_cols:
            if category not in summary:
                continue
            r = self.summaryRow
            self.sheet["{}{}".format(get_column_letter(c), r)].value = category

            AddFillColor(self.sheet, "{}{}".format(get_column_letter(c), r), "FEFEBE")
            AddBorder(self.sheet, "{}{}".format(get_column_letter(c), r))
            r += 1
            for item in summary[category]:
                self.sheet["{}{}".format(get_column_letter(c), r)].value = item
                AddFillColor(
                    self.sheet, "{}{}".format(get_column_letter(c), r), "FEFEBE"
                )
                AddBorder(self.sheet, "{}{}".format(get_column_letter(c), r))
                c += 1
                value = summary[category][item]
                set_number_format(self.sheet, c, r)
                self.sheet["{}{}".format(get_column_letter(c), r)].value = value
                AddFillColor(
                    self.sheet, "{}{}".format(get_column_letter(c), r), "FEFEBE"
                )
                AddBorder(self.sheet, "{}{}".format(get_column_letter(c), r))
                c -= 1
                r += 1
            c += 2

    def __init__(self, sheet):
        super(basicProcessor, self).__init__(sheet)
        self.summaryRow = self.getSummaryRow()

In [ ]:
class basicProcessorWithDoses(basicProcessor):

    summary_cols = ["Genders", "Age Ranges", "Ethnicity", "Race", "Zip Codes", "Doses"]

    @staticmethod
    def CanProcess(sheet):
        for col in range(1, 20):
            celldata = sheet["{}{}".format(get_column_letter(col), 1)].value
            if celldata is not None:
                if celldata.lower().strip() == "dose":
                    return basicProcessor.CanProcess(sheet)
        return False

    def getExtraSummary(self, rows):
        return {"Doses": tally(rows, "dose")}

In [ ]:
class basicProcessorWithCholestech(basicProcessor):

    summary_cols = [
        "Genders",
        "Age Ranges",
        "Ethnicity",
        "Race",
        "Zip Codes",
        "Doses",
        "Flu",
        "Blood Pressure",
        "Cholestech",
        "COVID Kits",
        "Outreach",
    ]

    @staticmethod
    def CanProcess(sheet):
        for col in range(1, 20):
            celldata = sheet["{}{}".format(get_column_letter(col), 1)].value
            if celldata is not None:
                if celldata.lower().strip() == "cholestech":
                    return basicProcessor.CanProcess(sheet)
        return False

    def getExtraSummary(self, rows):
        return {
            "Doses": tally(rows, "Doses"),
            "Flu": tally(rows, "Flu"),
            "Blood Pressure": tally(rows, "Blood Pressure"),
            "Cholestech": tally(rows, "Cholestech"),
            "COVID Kits": tally(rows, "COVID Kits"),
            "Outreach": tally(rows, "Outreach"),
        }

In [ ]:
def parseDate(title):
    try:
        return datetime.strptime(title.strip().split(" ").pop(), "%m.%d.%y")
    except:
        pass
    try:
        return datetime.strptime(title.strip().split(" ").pop(0), "%m.%d.%y")
    except:
        pass
    try:
        return datetime.strptime(title.strip().split(" ").pop(0), "%m-%d-%y")
    except:
        pass
    raise Exception("BAD: " + title)

In [ ]:
def setCell(sheet, col, row, data):
    print(sheet, col, type(col), row, type(row), data, type(data))
    sheet["{}{}".format(get_column_letter(col), row)].value = data


def styleCell(sheet, col, row):
    AddFillColor(sheet, "{}{}".format(get_column_letter(col), row), "FEFEBE")
    AddBorder(sheet, "{}{}".format(get_column_letter(col), row))


class ByMonthSummaryProcessor:
    summary_cols = ["Genders", "Age Ranges", "Ethnicity", "Race", "Zip Codes", "Doses"]

    def __init__(self, sheet, xfile):
        self.sheet = sheet
        self.xfile = xfile

    @staticmethod
    def CanProcess(sheet):
        return sheet.title == "By Month Summary"

    def clearSummary(self):
        self.sheet.delete_rows(1, self.sheet.max_row + 1)

    def getSummary(self):
        master_summary = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
        for sheet in self.xfile.worksheets:
            if "summary" in sheet.title.lower():
                continue
            processor = getProcessor(sheet, self.xfile)
            if processor is None:
                continue
            date = parseDate(sheet.title)
            summary = processor.getSummary()
            for category in summary:
                for item in summary[category]:
                    print(category, item, summary[category][item])
                    master_summary[date.strftime("%B %Y")][category][item] += summary[
                        category
                    ][item]

        return master_summary

    def updateSummary(self, summary):
        r = 1
        c = 1
        next_row_r = 1
        mr = 1
        for month_name in summary:
            print(month_name)
            c = 1
            r = mr
            setCell(self.sheet, c, r, month_name)
            styleCell(self.sheet, c, r)
            r += 1
            c += 1
            orginal_r = r
            for category in self.summary_cols:
                if category not in summary[month_name]:
                    continue
                r = orginal_r
                setCell(self.sheet, c, r, category)
                styleCell(self.sheet, c, r)
                r += 1
                mr = max(r, mr)
                for item in summary[month_name][category]:
                    setCell(self.sheet, c, r, item)
                    styleCell(self.sheet, c, r)
                    c += 1

                    value = summary[month_name][category][item]
                    set_number_format(self.sheet, c, r)
                    setCell(self.sheet, c, r, value)
                    styleCell(self.sheet, c, r)
                    c -= 1
                    r += 1
                    mr = max(r, mr)
                    if r > next_row_r:
                        next_row_r = r + 10
                c += 2
            # spacing between months
            r = mr
            r += 10

In [ ]:
class GrandSummaryProcessor:
    summary_cols = ["Genders", "Age Ranges", "Ethnicity", "Race", "Zip Codes", "Doses"]

    def __init__(self, sheet, xfile):
        self.sheet = sheet
        self.xfile = xfile

    @staticmethod
    def CanProcess(sheet):
        return sheet.title == "Grand Summary"

    def clearSummary(self):
        self.sheet.delete_rows(1, self.sheet.max_row + 1)

    def getSummary(self):
        master_summary = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
        for sheet in self.xfile.worksheets:
            if "summary" in sheet.title.lower():
                continue
            processor = getProcessor(sheet, self.xfile)
            if processor is None:
                continue
            date = parseDate(sheet.title)
            summary = processor.getSummary()
            print(summary)
            for category in summary:
                for item in summary[category]:
                    master_summary[date.strftime("%Y")][category][item] += summary[
                        category
                    ][item]

        return master_summary

    def updateSummary(self, summary):
        r = 1
        c = 1
        next_row_r = 1

        for year_name in summary:
            print(year_name)
            c = 1
            r = next_row_r
            setCell(self.sheet, c, r, year_name)
            styleCell(self.sheet, c, r)

            r += 1
            c += 1
            mr = r
            orginal_r = r
            for category in self.summary_cols:
                if category not in summary[year_name]:
                    continue
                r = orginal_r
                setCell(self.sheet, c, r, category)
                styleCell(self.sheet, c, r)
                r += 1
                mr = max(r, mr)
                for item in summary[year_name][category]:
                    setCell(self.sheet, c, r, item)
                    styleCell(self.sheet, c, r)
                    c += 1

                    value = summary[year_name][category][item]
                    set_number_format(self.sheet, c, r)
                    setCell(self.sheet, c, r, value)
                    styleCell(self.sheet, c, r)
                    c -= 1
                    r += 1
                    mr = max(r, mr)
                    if r > next_row_r:
                        next_row_r = r
                c += 2
            # spacing between months
            r = mr
            r += 10

In [ ]:
def getProcessor(sheet, xfile):
    print("Loading WS:", sheet.title)
    if sheet.title in ["By Month Summary", "Grand Summary"]:
        return None
    if sheet.title.lower().startswith("summary"):
        return None

    if basicProcessorWithDoses.CanProcess(sheet):
        return basicProcessorWithDoses(sheet)

    if basicProcessorWithDoses.CanProcess(sheet):
        return basicProcessorWithDoses(sheet)

    if basicProcessor.CanProcess(sheet):
        return basicProcessor(sheet)
    return None

In [ ]:
print("Loading '{}'".format(input_filename))

ifn = Path(input_filename)
if os.path.exists(input_filename):
    xfile = openpyxl.load_workbook(input_filename, data_only=True)

    if "By Month Summary" in xfile:
        del xfile["By Month Summary"]
    if "Grand Summary" in xfile:
        del xfile["Grand Summary"]
    for sheet in xfile.worksheets:
        processor = getProcessor(sheet, xfile)
        if processor is None:
            print(sheet, "Failed getting processor")
            continue
        processor.clearSummary()

    clean_output_filename = f"{ifn.stem}_cleaned.xlsx"
    xfile.save(clean_output_filename)
    m_summ = ByMonthSummaryProcessor(
        get_sheet_by_name(xfile, "By Month Summary", create=True), xfile
    )
    m_summ_summ = m_summ.getSummary()
    m_summ.updateSummary(m_summ_summ)

    g_summ = GrandSummaryProcessor(
        get_sheet_by_name(xfile, "Grand Summary", create=True), xfile
    )
    g_summ_summ = g_summ.getSummary()
    g_summ.updateSummary(g_summ_summ)

    for sheet in xfile.worksheets:
        processor = getProcessor(sheet, xfile)
        if processor is None:
            print(sheet, "Failed getting processor")
            continue
        processor.clearSummary()
        print("\t", "Building Summary")
        summary = processor.getSummary()

        print("\t\tSpotCheck data")
        for element in summary:
            print("\t\t", element)
            for item in summary[element]:
                print("\t\t\t\t", item, "=", summary[element][item])

        print("\t", "Updating Summary")
        processor.updateSummary(summary)
        print()
    output_filename = f"{ifn.stem}_updated.xlsx"

    print(f'Saving as "{output_filename}"')
    xfile.save(output_filename)
    print("done")
else:
    print("File Missing, please program next to data file")
    input()

In [ ]:
raise Exception("Stop")

In [ ]:
master_summary = getProcessor(
    get_sheet_by_name(xfile, "Okemos Schools 01.28.22"), xfile
).getSummary()
master_summary

# Debugging code below


In [ ]:
xfile = openpyxl.load_workbook(input_filename, data_only=True)

In [ ]:
xfile.close()

In [ ]:
master_summary = getProcessor(xfile.worksheets[8], xfile).getSummary()

In [ ]:
master_summary

In [ ]:
a = getProcessor(xfile.worksheets[1])
a

In [ ]:
a.getHeaders()

In [ ]:
a.getSummary()